## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Domino Data Lab Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. You need to make sure that these datasets are uploaded and present in the same directory as this notebook file, since we will be importing these files in Spark and create the following tables under the __`country_club`__ database using Spark SQL.

1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into Spark to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the Domino platform, but for the final deliverable, please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

# Checking Existence of Spark Environment Variables

Make sure your notebook is loaded using a PySpark Workspace. If you open up a regular Jupyter workspace the following variables might not exist

In [1]:
spark

In [2]:
sqlContext

### Run the following if you failed to open a notebook in the PySpark Workspace

This will work assuming you are using Spark in the cloud on domino or you might need to configure with your own spark instance if you are working offline

In [3]:
if 'sc' not in locals():
    from pyspark.context import SparkContext
    from pyspark.sql.context import SQLContext
    from pyspark.sql.session import SparkSession
    
    sc = SparkContext()
    sqlContext = SQLContext(sc)
    spark = SparkSession(sc)

# Create a utility function to run SQL commands

Instead of typing the same python functions repeatedly, we build a small function where you can just pass your query to get results.

- Remember we are using Spark SQL in PySpark
- We can't run multiple SQL statements in one go (no semi-colon ';' separated SQL statements)
- We can run multi-line SQL queries (but still has to be a single statement)

In [1]:
def run_sql(statement):
    try:
        result = sqlContext.sql(statement)
    except Exception as e:
        print(e.desc, '\n', e.stackTrace)
        return
    return result

# Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [2]:
run_sql('drop database if exists country_club cascade')
run_sql('create database country_club')
dbs = run_sql('show databases')
dbs.toPandas()

,databaseName
0,country_club
1,default


# Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you.

To get started, first make sure you have already uploaded the three CSV files and they are present in the same directory as the notebook.

Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [3]:
# File location and type
file_location_bookings = "./Bookings.csv"
file_location_facilities = "./Facilities.csv"
file_location_members = "./Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [4]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
 |-- bookid: integer (nullable = true)
 |-- facid: integer (nullable = true)
 |-- memid: integer (nullable = true)
 |-- starttime: timestamp (nullable = true)
 |-- slots: integer (nullable = true)

Facilities Schema
root
 |-- facid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- membercost: double (nullable = true)
 |-- guestcost: double (nullable = true)
 |-- initialoutlay: integer (nullable = true)
 |-- monthlymaintenance: integer (nullable = true)

Members Schema
root
 |-- memid: integer (nullable = true)
 |-- surname: string (nullable = true)
 |-- firstname: string (nullable = true)
 |-- address: string (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- telephone: string (nullable = true)
 |-- recommendedby: integer (nullable = true)
 |-- joindate: timestamp (nullable = true)



# Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [5]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

### Refresh tables and check them

In [6]:
run_sql('use country_club')
run_sql('REFRESH table bookings')
run_sql('REFRESH table facilities')
run_sql('REFRESH table members')
tbls = run_sql('show tables')
tbls.toPandas()

,database,tableName,isTemporary
0,country_club,bookings,False
1,country_club,facilities,False
2,country_club,members,False


# Test a sample SQL query

__Note:__ You can use multi-line SQL queries (but still a single statement) as follows

In [7]:
result = run_sql('''
                    SELECT * 
                    FROM bookings 
                    LIMIT 3
                 ''')
result.toPandas()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2


# Your Turn: Solve the following questions with Spark SQL

- Make use of the `run_sql(...)` function as seen in the previous example
- You can write multi-line SQL queries but it has to be a single statement (no use of semi-colons ';')
- Make use of the `toPandas()` function as depicted in the previous example to display the query results

#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [8]:
result = run_sql('''
                  select * from facilities 
                 ''')
result.toPandas()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000
5,5,Massage Room 2,9.9,80.0,4000,3000
6,6,Squash Court,3.5,17.5,5000,80
7,7,Snooker Table,0.0,5.0,450,15
8,8,Pool Table,0.0,5.0,400,15


In [9]:
result = run_sql('''
                   select * from facilities where membercost = 0
                 ''')
result.toPandas()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,2,Badminton Court,0.0,15.5,4000,50
1,3,Table Tennis,0.0,5.0,320,10
2,7,Snooker Table,0.0,5.0,450,15
3,8,Pool Table,0.0,5.0,400,15


####  Q2: How many facilities do not charge a fee to members?

In [10]:
result = run_sql('''
                   select count(*) from facilities where membercost = 0
                 ''')
result.toPandas()

,count(1)
0,4


#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [14]:
#

In [11]:
result = run_sql('''
                   select facid, name, membercost, monthlymaintenance from facilities where membercost < 0.2 * monthlymaintenance
                 ''')
result.toPandas()

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,2,Badminton Court,0.0,50
3,3,Table Tennis,0.0,10
4,4,Massage Room 1,9.9,3000
5,5,Massage Room 2,9.9,3000
6,6,Squash Court,3.5,80
7,7,Snooker Table,0.0,15
8,8,Pool Table,0.0,15


#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [16]:
result = run_sql('''
                   select * from facilities where facid in (1, 5)
                 ''')
result.toPandas()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25.0,8000,200
1,5,Massage Room 2,9.9,80.0,4000,3000


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [17]:
result = run_sql('''
                   SELECT *, 
CASE
    WHEN monthlymaintenance < 100 THEN "cheap"
    ELSE "expensive"
END AS priceclass
from facilities
                 ''')
result.toPandas()

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance,priceclass
0,0,Tennis Court 1,5.0,25.0,10000,200,expensive
1,1,Tennis Court 2,5.0,25.0,8000,200,expensive
2,2,Badminton Court,0.0,15.5,4000,50,cheap
3,3,Table Tennis,0.0,5.0,320,10,cheap
4,4,Massage Room 1,9.9,80.0,4000,3000,expensive
5,5,Massage Room 2,9.9,80.0,4000,3000,expensive
6,6,Squash Court,3.5,17.5,5000,80,cheap
7,7,Snooker Table,0.0,5.0,450,15,cheap
8,8,Pool Table,0.0,5.0,400,15,cheap


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [21]:
result = run_sql('''
                  select * from members limit 5
                 ''')
result.toPandas()

,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,NaN,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,NaN,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,NaN,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,NaN,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1.0,2012-07-03 10:25:05


In [25]:
result = run_sql('''
                    select max(joindate) from members
                 ''')
result.toPandas()

,max(joindate)
0,2012-09-26 18:08:45


In [27]:
result = run_sql('''
                select firstname, surname from members where joindate = (select max(joindate) from members)
                 ''')
result.toPandas()

,firstname,surname
0,Darren,Smith


####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [29]:
result = run_sql('''
                   select * from bookings limit 5
                 ''')
result.toPandas()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [48]:
result = run_sql('''
                   SELECT bookings.memid, firstname, surname, address, zipcode, telephone, recommendedby, joindate
  FROM bookings JOIN members
    ON bookings.memid = members.memid 
    limit 5
                 ''')
result.toPandas()

,memid,firstname,surname,address,zipcode,telephone,recommendedby,joindate
0,1,Darren,Smith,"8 Bloomsbury Close, Boston",4321,555-555-5555,None,2012-07-02 12:02:05
1,1,Darren,Smith,"8 Bloomsbury Close, Boston",4321,555-555-5555,None,2012-07-02 12:02:05
2,0,GUEST,GUEST,GUEST,0,(000) 000-0000,None,2012-07-01 00:00:00
3,1,Darren,Smith,"8 Bloomsbury Close, Boston",4321,555-555-5555,None,2012-07-02 12:02:05
4,1,Darren,Smith,"8 Bloomsbury Close, Boston",4321,555-555-5555,None,2012-07-02 12:02:05


In [69]:
result = run_sql('''
                   SELECT bookings.memid, firstname, surname, name, 
                   CASE 
                   WHEN bookings.memid = 0 THEN facilities.guestcost*slots
                   ELSE facilities.membercost*slots
                   END as cost
                   FROM bookings JOIN members
                   ON bookings.memid = members.memid 
                   join facilities on bookings.facid = facilities.facid 
                   limit 5
                 ''')
result.toPandas()

,memid,firstname,surname,name,cost
0,1,Darren,Smith,Table Tennis,0.0
1,1,Darren,Smith,Massage Room 1,19.8
2,0,GUEST,GUEST,Squash Court,35.0
3,1,Darren,Smith,Snooker Table,0.0
4,1,Darren,Smith,Pool Table,0.0


#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [38]:
#gets the bookings on the right day
result = run_sql('''
                   select * from bookings where starttime >= '2012-09-14 00:00:00' and starttime < '2012-09-14 23:59:59' limit 5
                 ''')
result.toPandas()

,bookid,facid,memid,starttime,slots
0,2916,0,6,2012-09-14 08:00:00,3
1,2917,0,17,2012-09-14 10:00:00,3
2,2918,0,5,2012-09-14 12:30:00,3
3,2919,0,3,2012-09-14 14:00:00,3
4,2920,0,0,2012-09-14 16:00:00,3


In [72]:
#memid 0 is guest
result = run_sql('''
                   SELECT bookings.memid, firstname, surname, name, 
                   CASE 
                   WHEN bookings.memid = 0 THEN facilities.guestcost*slots
                   ELSE facilities.membercost*slots
                   END as cost
                   FROM bookings JOIN members
                   ON bookings.memid = members.memid 
                   join facilities on bookings.facid = facilities.facid 
                   WHERE cost > 30
                 ''')
result.toPandas()

cannot resolve '`cost`' given input columns: [country_club.members.surname, country_club.bookings.bookid, country_club.bookings.starttime, country_club.members.address, country_club.members.joindate, country_club.members.firstname, country_club.facilities.facid, country_club.facilities.name, country_club.bookings.facid, country_club.bookings.memid, country_club.facilities.guestcost, country_club.bookings.slots, country_club.members.zipcode, country_club.facilities.initialoutlay, country_club.members.recommendedby, country_club.members.telephone, country_club.facilities.monthlymaintenance, country_club.facilities.membercost, country_club.members.memid]; line 10 pos 25;
'Project ['bookings.memid, 'firstname, 'surname, 'name, CASE WHEN ('bookings.memid = 0) THEN ('facilities.guestcost * 'slots) ELSE ('facilities.membercost * 'slots) END AS cost#466]
+- 'Filter ('cost > 30)
   +- Join Inner, (facid#153 = facid#162)
      :- Join Inner, (memid#154 = memid#211)
      :  :- SubqueryAlias `cou

AttributeError: 'NoneType' object has no attribute 'toPandas'

In [80]:
#select firstname, surname from members where joindate = (select max(joindate) from members)
result = run_sql('''
                   SELECT bookings.memid, firstname, surname, name
                   FROM bookings JOIN members
                   ON bookings.memid = members.memid 
                   join facilities on bookings.facid = facilities.facid 
                   where starttime >= '2012-09-14 00:00:00' and starttime < '2012-09-14 23:59:59'
                   limit 5
                 ''')
result.toPandas()

,memid,firstname,surname,name
0,6,Burton,Tracy,Tennis Court 1
1,17,David,Pinker,Tennis Court 1
2,5,Gerald,Butters,Tennis Court 1
3,3,Tim,Rownam,Tennis Court 1
4,0,GUEST,GUEST,Tennis Court 1


In [83]:
#select firstname, surname from members where joindate = (select max(joindate) from members)
result = run_sql('''
                   SELECT bookings.memid, firstname, surname, name
                   FROM bookings JOIN members
                   ON bookings.memid = members.memid 
                   join facilities on bookings.facid = facilities.facid 
                   where starttime >= '2012-09-14 00:00:00' and starttime < '2012-09-14 23:59:59' and 30 <  
                   CASE
                   WHEN bookings.memid = 0 THEN facilities.guestcost*slots
                   ELSE facilities.membercost*slots
                   END
                 ''')
result.toPandas()

,memid,firstname,surname,name
0,0,GUEST,GUEST,Tennis Court 1
1,0,GUEST,GUEST,Tennis Court 1
2,0,GUEST,GUEST,Tennis Court 2
3,0,GUEST,GUEST,Tennis Court 2
4,0,GUEST,GUEST,Massage Room 1
5,0,GUEST,GUEST,Massage Room 1
6,13,Jemima,Farrell,Massage Room 1
7,0,GUEST,GUEST,Massage Room 1
8,0,GUEST,GUEST,Massage Room 2
9,0,GUEST,GUEST,Squash Court


In [85]:
#select firstname, surname from members where joindate = (select max(joindate) from members)
result = run_sql('''
                   SELECT bookings.memid, firstname, surname, name, CASE 
                                                                    WHEN bookings.memid = 0 THEN facilities.guestcost*slots
                                                                    ELSE facilities.membercost*slots
                                                                    END as cost
                   
                   FROM bookings JOIN members
                   ON bookings.memid = members.memid 
                   join facilities on bookings.facid = facilities.facid 
                   
                   WHERE starttime >= '2012-09-14 00:00:00' and starttime < '2012-09-14 23:59:59' and 30 < CASE
                                                                                                           WHEN bookings.memid = 0 THEN facilities.guestcost*slots
                                                                                                           ELSE facilities.membercost*slots
                                                                                                           END
                    
                   ORDER BY cost DESC
                 ''')
result.toPandas()

,memid,firstname,surname,name,cost
0,0,GUEST,GUEST,Massage Room 2,320.0
1,0,GUEST,GUEST,Massage Room 1,160.0
2,0,GUEST,GUEST,Massage Room 1,160.0
3,0,GUEST,GUEST,Massage Room 1,160.0
4,0,GUEST,GUEST,Tennis Court 2,150.0
5,0,GUEST,GUEST,Tennis Court 1,75.0
6,0,GUEST,GUEST,Tennis Court 1,75.0
7,0,GUEST,GUEST,Tennis Court 2,75.0
8,0,GUEST,GUEST,Squash Court,70.0
9,13,Jemima,Farrell,Massage Room 1,39.6


#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [120]:
#select firstname, surname from members where joindate = (select max(joindate) from members)
result = run_sql('''

                   SELECT nameTable.memid, firstname, surname, nameTable.name, CASE 
                                                                              WHEN nameTable.memid = 0 THEN guestcost*slots
                                                                              ELSE membercost*slots
                                                                              END as cost
                    from (select name, bookings.memid, guestcost, membercost, slots, starttime from bookings join facilities on bookings.facid = facilities.facid) as nameTable
                    join members on members.memid = nameTable.memid

                   
                   WHERE starttime >= '2012-09-14 00:00:00' and starttime < '2012-09-14 23:59:59' and 30 < CASE
                                                                                                           WHEN nameTable.memid = 0 THEN guestcost*slots
                                                                                                           ELSE membercost*slots
                                                                                                           END
                    
                   ORDER BY cost DESC
                 ''')
result.toPandas()

,memid,firstname,surname,name,cost
0,0,GUEST,GUEST,Massage Room 2,320.0
1,0,GUEST,GUEST,Massage Room 1,160.0
2,0,GUEST,GUEST,Massage Room 1,160.0
3,0,GUEST,GUEST,Massage Room 1,160.0
4,0,GUEST,GUEST,Tennis Court 2,150.0
5,0,GUEST,GUEST,Tennis Court 1,75.0
6,0,GUEST,GUEST,Tennis Court 1,75.0
7,0,GUEST,GUEST,Tennis Court 2,75.0
8,0,GUEST,GUEST,Squash Court,70.0
9,13,Jemima,Farrell,Massage Room 1,39.6


In [119]:
#select firstname, surname from members where joindate = (select max(joindate) from members)
result = run_sql('''
                   SELECT nameTable.memid, firstname, surname, nameTable.name, CASE 
                                                                              WHEN nameTable.memid = 0 THEN guestcost*slots
                                                                              ELSE membercost*slots
                                                                              END as cost
                    from (select name, bookings.memid, guestcost, membercost, slots, starttime from bookings join facilities on bookings.facid = facilities.facid) as nameTable
                    join members on members.memid = nameTable.memid
                   
                 ''')
result.toPandas()

,memid,firstname,surname,name,cost
0,1,Darren,Smith,Table Tennis,0.0
1,1,Darren,Smith,Massage Room 1,19.8
2,0,GUEST,GUEST,Squash Court,35.0
3,1,Darren,Smith,Snooker Table,0.0
4,1,Darren,Smith,Pool Table,0.0
5,1,Darren,Smith,Pool Table,0.0
6,2,Tracy,Smith,Tennis Court 1,15.0
7,2,Tracy,Smith,Tennis Court 1,15.0
8,3,Tim,Rownam,Massage Room 1,19.8
9,0,GUEST,GUEST,Massage Room 1,160.0


In [114]:
#select firstname, surname from members where joindate = (select max(joindate) from members)
result = run_sql('''
                   SELECT members.memid, firstname, surname, nameTable.name, CASE 
                                                                              WHEN nameTable.memid = 0 THEN guestcost*slots
                                                                              ELSE membercost*slots
                                                                              END as cost
                    from (select name, bookings.memid, guestcost, membercost, slots from bookings join facilities on bookings.facid = facilities.facid) as nameTable
                    join members on members.memid = nameTable.memid
                   
                 ''')
result.toPandas()

,memid,firstname,surname,name,cost
0,1,Darren,Smith,Table Tennis,0.0
1,1,Darren,Smith,Massage Room 1,19.8
2,0,GUEST,GUEST,Squash Court,35.0
3,1,Darren,Smith,Snooker Table,0.0
4,1,Darren,Smith,Pool Table,0.0
5,1,Darren,Smith,Pool Table,0.0
6,2,Tracy,Smith,Tennis Court 1,15.0
7,2,Tracy,Smith,Tennis Court 1,15.0
8,3,Tim,Rownam,Massage Room 1,19.8
9,0,GUEST,GUEST,Massage Room 1,160.0


In [105]:
#select firstname, surname from members where joindate = (select max(joindate) from members)
result = run_sql('''
                   SELECT members.memid, firstname, surname, nameTable.name
                    from (select name, bookings.memid from bookings join facilities on bookings.facid = facilities.facid) as nameTable
                    join members on members.memid = nameTable.memid
                   
                 ''')
result.toPandas()

,memid,firstname,surname,name
0,1,Darren,Smith,Table Tennis
1,1,Darren,Smith,Massage Room 1
2,0,GUEST,GUEST,Squash Court
3,1,Darren,Smith,Snooker Table
4,1,Darren,Smith,Pool Table
5,1,Darren,Smith,Pool Table
6,2,Tracy,Smith,Tennis Court 1
7,2,Tracy,Smith,Tennis Court 1
8,3,Tim,Rownam,Massage Room 1
9,0,GUEST,GUEST,Massage Room 1


In [103]:
#

In [98]:
#select firstname, surname from members where joindate = (select max(joindate) from members)
result = run_sql('''
                   
                    select name from bookings join facilities on bookings.facid = facilities.facid
                   
                   
                 ''')
result.toPandas()

,name
0,Table Tennis
1,Massage Room 1
2,Squash Court
3,Snooker Table
4,Pool Table
5,Pool Table
6,Tennis Court 1
7,Tennis Court 1
8,Massage Room 1
9,Massage Room 1


In [89]:
#select firstname, surname from members where joindate = (select max(joindate) from members)
result = run_sql('''
                   SELECT 
                   CASE
                   WHEN bookings.memid = 0 THEN facilities.guestcost*slots
                   ELSE facilities.membercost*slots
                   END as cost
                   FROM bookings join facilities on bookings.facid = facilities.facid
                    
                 ''')
result.toPandas()

,cost
0,0.0
1,19.8
2,35.0
3,0.0
4,0.0
5,0.0
6,15.0
7,15.0
8,19.8
9,160.0


#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [132]:
#select firstname, surname from members where joindate = (select max(joindate) from members)
#SUM(nameTable.cost) as revenue, (in select)
result = run_sql('''

                   SELECT nameTable.name, sum(cost) as revenue
                   FROM (select name,  CASE 
                                       WHEN bookings.memid = 0 THEN guestcost*slots
                                       ELSE membercost*slots
                                       END as cost
                          
                          from bookings join facilities on bookings.facid = facilities.facid) as nameTable
                   GROUP BY nameTable.name
                   HAVING revenue < 1000
                   ORDER BY revenue DESC
                 ''')
result.toPandas()

,name,revenue
0,Pool Table,270.0
1,Snooker Table,240.0
2,Table Tennis,180.0


In [124]:
#select firstname, surname from members where joindate = (select max(joindate) from members)
result = run_sql('''

                   select name,  CASE 
                                       WHEN bookings.memid = 0 THEN guestcost*slots
                                       ELSE membercost*slots
                                       END as cost
                          
                          from bookings join facilities on bookings.facid = facilities.facid

                 ''')
result.toPandas()

,name,cost
0,Table Tennis,0.0
1,Massage Room 1,19.8
2,Squash Court,35.0
3,Snooker Table,0.0
4,Pool Table,0.0
5,Pool Table,0.0
6,Tennis Court 1,15.0
7,Tennis Court 1,15.0
8,Massage Room 1,19.8
9,Massage Room 1,160.0
